# Numba, temps, mémoire et matrices sparse 

      Joseph Salmon : joseph.salmon@umontpellier.fr


# Numba: compilation "Just in Time" (jit)

Numba convertit les fonctions Python en code machine optimisé au moment de l'exécution à l'aide de la bibliothèque de compilateur LLVM standard.
Les algorithmes numériques compilés par Numba en Python peuvent alors approcher les vitesses de C ou de FORTRAN, la où les boucles classiques, comme en R et matlab, peuvent être un peu lente.

https://numba.pydata.org/

In [ ]:
from numba import jit
import numpy as np
import time
from numba import jit
import numpy as np
import time

# Temps et calcul: exemple d'utilisation de `%timeit`
Remarque pour d'autres commandes *magiques* comme %timeit, %matplotlib, %autoreload: 

In [ ]:
n  = 1000
val = 5.4

In [ ]:
%timeit a = np.empty(n); a.fill(val)

In [ ]:
%timeit a=np.empty(n); a[:]=val

In [ ]:
%timeit a = np.full((n,), val)

In [ ]:
%timeit a=np.ones(n)*val

In [ ]:
%timeit a=np.repeat(val,n)

# Debogage : 
https://davidhamann.de/2017/04/22/debugging-jupyter-notebooks/
utiliser  `import pdb; pdb.set_trace()` pour rentrer dans un code et requêter les informations des valeurs. Enfin pour continuer le code entre `c` et la touche `enter`.

On peut consulter aussi:
https://www.codementor.io/stevek/advanced-python-debugging-with-pdb-g56gvmpfa.

In [ ]:
def add_to_life_universe_everything(x):
    answer = 42
    import pdb; pdb.set_trace()
    answer += x
    
    return answer

add_to_life_universe_everything(12)

Une autre manière de procéder est d'allumer le deboguer `pdb`. Une invite de commande se lance alors quand on a un soucis, et on peut alors reprendre la main voir ce qu'il se passe.

In [ ]:
%pdb

In [ ]:
def blobl_func(x):
    answer = 0
    for i in range(x,-1,-1):
        print(i)
        answer += 1 / i
    
    return answer

In [ ]:
blobl_func(4)

# Notion de tests:
https://semaphoreci.com/community/tutorials/testing-python-applications-with-pytest

# Exemple 1: Méthode de Monte Carlo pour approcher $\pi$

In [ ]:
@jit(nopython=True)
def monte_carlo_pi(n_samples=1000):
    acc = 0
    for sample in range(n_samples):
        vec = np.random.rand(2)
        if np.linalg.norm(vec) < 1.:
            acc += 1
    return 4.0 * acc / n_samples 

In [ ]:

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
monte_carlo_pi(n_samples=10000000)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
monte_carlo_pi(n_samples=1000)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

In [ ]:
def go_slow(a):  # Function is compiled and runs in machine code
    trace = 0
    for i in range(a.shape[0]):
        trace += np.tanh(a[i, i])
    return trace
all_n_samples = [1000, 5000, 10000]
t0 = []
t1 = []
t2 = []

for n_samples in all_n_samples:
    print(n_samples)
    x = np.arange(n_samples ** 2).reshape(n_samples, n_samples)

    @jit(nopython=True)
    def go_fast(a):  # Function is compiled and runs in machine code
        trace = 0
        for i in range(a.shape[0]):
            trace += np.tanh(a[i, i])
        return trace
    # COMPILATION INCLUSE!
    start = time.time()
    go_fast(x)
    end = time.time()
    t0.append(end - start)
    print("Elapsed (with compilation) = %s" % (end - start))
    # COMPILATION NON INCLUSE, EXECUTER DEPUIS LE CACHE 
    start = time.time()
    go_fast(x)
    end = time.time()
    t1.append(end - start)
    print("Elapsed (after compilation) = %s" % (end - start))
    # VANILLA PYTHON
    start = time.time()
    go_slow(x)
    end = time.time()
    t2.append(end - start)
    print("Elapsed (vanilla) = %s" % (end - start))

    
t0 = np.array(t0)
t1 = np.array(t1)
t2 = np.array(t2)


print(all_n_samples)
print("Améliorations en pourcentage par rapport au code vanilla")
print((t0 - t2)/t2 *100)
print((t1 - t2)/t2*100)

# Objectif: descente de gradient avec/sans numba.

In [ ]:
n_samples = 1000
n_features = 500
n_iterations = 2000

X = np.random.randn(n_samples, n_features)
y = np.random.randn(n_samples)
y[n_samples // 2:] = 0

w = np.zeros(n_features)  # init = 0

In [ ]:
@jit(nopython=True)
# Function is compiled and runs in machine code
def gradient(X, y, w, step_size=0.01,  max_iter=1000):
    """Gradient descent with constant step size."""
    for k in range(max_iter):
        w -=  step_size * (X.T.dot(X.dot(w) - y))
    return w

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
gradient(X, y, w)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
gradient(X, y, w)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

# Exemple: Régression logistique

In [ ]:
y = np.random.randint(2, size=n_samples) *2 -1
print(y)
w = np.zeros(n_features)  # init = 0

In [ ]:
def logistic_regression_no_jit(y, X, w, iterations=1000):
    for i in range(iterations):
        w -= np.dot(((1.0 / (1.0 + np.exp(-y * np.dot(X, w))) - 1.0) * y), X)
    return w

In [ ]:
start = time.time()
logistic_regression_no_jit(y, X, w, iterations=n_iterations )
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))


In [ ]:
@jit(nopython=True)
def logistic_regression(y, X, w, iterations=1000):
    for i in range(iterations):
        w -= np.dot(((1.0 / (1.0 + np.exp(-y * np.dot(X, w))) - 1.0) * y), X)
    return w

In [ ]:
# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
logistic_regression(y, X, w, iterations=n_iterations)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
logistic_regression(y, X, w, iterations=n_iterations)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

In [ ]:
%pdb off

# Matrices Creuses et mémoire
http://scipy-lectures.org/advanced/scipy_sparse/introduction.html#why-sparse-matrices

https://rushter.com/blog/scipy-sparse-matrices/

http://cmdlinetips.com/2018/03/sparse-matrices-in-python-with-scipy/

Un cadre classique d'application des matrices creuses est le cadre des graphes: bien que le nombre de noeuds puissent être énorme, chaque noeud d'un graphe n'est en général pas relié à tous ces voisins. Si on représente un grapha par ca matrice d'adjacence:

## Définition: *matrice d'adjacence*:
Supposons que $G=(V,E)$ est un graphe, où $\left|V\right|=n$.
Supposons que les sommets de $G$ sont numérotés arbitrairement $v_1,\ldots,v_n$. 
La matrice d'adjacence $A$ de $G$ est la matrice $n \times n$ de terme générale:

$$a_{ij}=\left\{\begin{array}{rl}
	1 & \mbox{si } (v_i,v_j) \in E \\
        0 & \mbox{sinon.}
\end{array}\right.$$


In [ ]:
import networkx as nx
G = nx.Graph()
G.add_edge('A', 'B', weight=4)
G.add_edge('B', 'D', weight=2)
G.add_edge('A', 'C', weight=3)
G.add_edge('C', 'D', weight=4)
G.add_edge('D', 'A', weight=2)

nx.draw(G, with_labels=True)

In [ ]:
A = nx.adjacency_matrix(G)
print(A.todense())

In [ ]:
nx.shortest_path(G, 'A', 'D', weight='weight')

# Définition : *matrice d'incidence*
Soit $G = (V,E)$ un graphe (non-orienté) à $n$ sommets, $V = [1,\dots,n] $, et $p$ arrêtes, $E = [1,\dot,p]$.
Le graphe peut être représenté par sa matrice d'incidence arrête-sommet $D^\top = \in \mathbb{R}^{p \times n}$ défini par
\begin{equation}
  (D^\top)_{e,v} =
  \begin{cases}
    + 1, & \text{si } v = \min(i,j) \\
    -1, & \text{si } v = \max(i,j) \\
    0, & \text{sinon}
  \end{cases}\enspace,
\end{equation}
où $e = \{i,j\}$.

# Définition : *matrice d'incidence*

The matrix $L=D D^\top$ is the so-called graph Laplacian of $G$


In [ ]:
D = nx.incidence_matrix(G, oriented=True).T
print(D.todense())

In [ ]:
import osmnx as ox
%matplotlib notebook
G = ox.graph_from_place('Montpellier, France', network_type='bike')

In [ ]:
ox.plot_graph(G)

In [ ]:
G.number_of_edges()

In [ ]:
G.number_of_nodes()

In [ ]:
D = nx.incidence_matrix(G, oriented=True).T

In [ ]:
D

In [ ]:
D.data.nbytes
# data_size = D.nbytes/(1024**2)
# print('Size of full matrix with zeros: '+ '%3.2f' %data_size + ' MB')

# Sparsité du graphe:

In [ ]:
print("Il a {0:.2} % d'arrêtes utlile pour représenter le graphe de la ville de Montpellier".format(100 * G.number_of_edges() / G.number_of_nodes() ** 2))

## Remarques : divers type de matrices creuses:

1. bsr_matrix: Block Sparse Row matrix
1. coo_matrix: COOrdinate format matrix
1. csc_matrix: Compressed Sparse Column matrix
1. csr_matrix: Compressed Sparse Row matrix
1. dia_matrix: Sparse matrix with DIAgonal storage
1. dok_matrix: Dictionary Of Keys based sparse matrix.
1. lil_matrix: Row-based linked list sparse matrix


Selon la nature et la structure des données, `csc_matrix` est plus efficace pour le `slicing` par colonne, alors que 
csr_matrix est plus efficace pour le cas ligne (en statistiques et en machine learning c'est souvent `css_matrix` qui sont le plus utilisées.